In [234]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import csv
import scipy.stats as st
from scipy.stats import linregress
import time
from datetime import date



In [235]:
#Study the covid data files
covid_dhss_path = "Data/COVID_VIC.csv"

#Read the Covid data
covid_dhss_data = pd.read_csv(covid_dhss_path)


In [236]:
#Checks to ensure the reading is successful
covid_dhss_data.head()


,diagnosis_date,Postcode,acquired,Localgovernmentarea
0,11/03/2020,3216.0,Travel overseas,Greater Geelong
1,8/03/2020,3942.0,Travel overseas,Mornington Peninsula
2,9/03/2020,3206.0,Travel overseas,Port Phillip
3,1/08/2020,3754.0,Contact with a confirmed case,Whittlesea
4,1/03/2020,3802.0,Travel overseas,Casey


In [237]:
covid_dhss_data.dtypes

diagnosis_date          object
Postcode               float64
acquired                object
Localgovernmentarea     object
dtype: object

In [238]:
#removing overseas and interstates cases from Localgovernmentarea
covid_dhss_local = covid_dhss_data[(covid_dhss_data["Localgovernmentarea"]!="Overseas") & (covid_dhss_data["Localgovernmentarea"]!="Interstate")]
covid_dhss_local



,diagnosis_date,Postcode,acquired,Localgovernmentarea
0,11/03/2020,3216.0,Travel overseas,Greater Geelong
1,8/03/2020,3942.0,Travel overseas,Mornington Peninsula
2,9/03/2020,3206.0,Travel overseas,Port Phillip
3,1/08/2020,3754.0,Contact with a confirmed case,Whittlesea
4,1/03/2020,3802.0,Travel overseas,Casey
...,...,...,...,...
20677,12/06/2021,3012.0,Contact with a confirmed case,Maribyrnong
20681,11/06/2021,3006.0,Contact with a confirmed case,Melbourne
20685,15/06/2021,3006.0,Contact with a confirmed case,Melbourne
20686,15/06/2021,3006.0,Contact with a confirmed case,Melbourne


In [239]:
#covid_dhss_local['Localgovernmentarea']=covid_dhss_local['Localgovernmentarea'].str.strip()
##covid_dhss_local[covid_dhss_local.Localgovernmentarea=='Melbourne ']
#other = pd.DataFrame({'Localgovernmentarea':metropolitan_lga})
#covid_dhss_local[covid_dhss_local.Localgovernmentarea=='Melbourne ']
#covid_dhss_local.isin(other)

In [240]:
#Just Removing other regional states
#Set up metropolitan local government area list
metropolitan_lga =['Banyule ','Hume ', 'Moreland ','Bayside ', 'Kingston ', 'Mornington Peninsula ', 'Boroondara ', 'Knox ', 'Nillumbik ', 'Brimbank ', 
                    'Manningham ', 'Port Phillip ', 'Cardinia ', 'Maribyrnong ', 'Stonnington ', 'Casey ', 'Maroondah ', 'Whitehorse ', 
                    'Darebin ', 'Melbourne ', 'Whittlesea ', 'Frankston ', 'Melton ', 'Wyndham ', 'Glen Eira ', 'Monash ', 'Yarra ', 'Greater Dandenong ', 
                    'Moonee Valley ', 'Yarra Ranges ','Hobsons Bay ']


#isin metropolitan_lga to set all the information stored under this variable will just be metropolitan_lga
covid_dhss_metro = covid_dhss_local[covid_dhss_local['Localgovernmentarea'].isin(metropolitan_lga)]                 
covid_dhss_metro
#print(covid_dhss_metro)

#Formatting diagnosis date
covid_dhss_metro['diagnosis_date'] = pd.to_datetime(covid_dhss_metro['diagnosis_date']).dt.strftime('%d-%m-%Y')
covid_dhss_metro




<ipython-input-240-77fadc988320>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covid_dhss_metro['diagnosis_date'] = pd.to_datetime(covid_dhss_metro['diagnosis_date']).dt.strftime('%d-%m-%Y')


,diagnosis_date,Postcode,acquired,Localgovernmentarea
1,03-08-2020,3942.0,Travel overseas,Mornington Peninsula
2,03-09-2020,3206.0,Travel overseas,Port Phillip
3,08-01-2020,3754.0,Contact with a confirmed case,Whittlesea
4,03-01-2020,3802.0,Travel overseas,Casey
5,28-01-2020,3150.0,Travel overseas,Monash
...,...,...,...,...
20677,06-12-2021,3012.0,Contact with a confirmed case,Maribyrnong
20681,06-11-2021,3006.0,Contact with a confirmed case,Melbourne
20685,15-06-2021,3006.0,Contact with a confirmed case,Melbourne
20686,15-06-2021,3006.0,Contact with a confirmed case,Melbourne


In [241]:
#Store the clean data into a new csv

covid_dhss_metro.to_csv (r'Data\covid_metro.csv', index = False)

In [244]:
#covid_metro_daily = covid_dhss_metro.groupby(['Postcode', 'diagnosis_date'])['Postcode'].count()
covid_metro_daily = covid_dhss_metro.groupby(['Postcode','diagnosis_date'], as_index=False).count()
covid_metro_daily

,Postcode,diagnosis_date,acquired,Localgovernmentarea
0,3000.0,04-01-2020,1,1
1,3000.0,04-04-2020,1,1
2,3000.0,04-06-2020,1,1
3,3000.0,06-03-2021,2,2
4,3000.0,07-03-2020,1,1
...,...,...,...,...
7512,3980.0,18-08-2020,2,2
7513,3981.0,17-08-2020,1,1
7514,3984.0,09-05-2020,1,1
7515,3984.0,09-10-2020,1,1


In [243]:
#Try to sort values
#covid_metro_daily.sort_values(['diagnosis_date'])

In [232]:
covid_metro_daily.to_csv (r'Data\covid_metro_daily.csv', index = False)

In [209]:
#lockdowndates
#covid_lockdown_path = "Data/lockdowndates.csv"

#Read the Covid data
#covid_lockdown_data = pd.read_csv(covid_lockdown_path)
#covid_lockdown_data

,Stage 4,Stage 3,Stage 2,Stage 1,Stage 0
0,02-08-2020,28-03-2020,26-10-2020,22-11-2020,27-11-2020
1,03-08-2020,29-03-2020,27-10-2020,23-11-2020,28-11-2020
2,04-08-2020,30-03-2020,28-10-2020,24-11-2020,29-11-2020
3,05-08-2020,31-03-2020,29-10-2020,25-11-2020,30-11-2020
4,06-08-2020,01-04-2020,30-10-2020,26-11-2020,01-12-2020
...,...,...,...,...,...
193,NaN,17-04-2021,NaN,NaN,NaN
194,NaN,18-04-2021,NaN,NaN,NaN
195,NaN,19-04-2021,NaN,NaN,NaN
196,NaN,20-04-2021,NaN,NaN,NaN
